In [1]:
import sys
sys.path.append("../")
from copy import deepcopy

import torch
import logging
from torch.utils.data import DataLoader

import transformers
from transformers import AutoTokenizer, AutoModel

from src.models.rnn_model import RNN
from src.models.learned_masking import (
    HighwayAugmenter,
    WeightedMaskClassificationLoss,
    HighwayAugmenterTrainer
)
from src.data.dataio import DataFiles, Dataset, truncate_fn

transformers.logging.set_verbosity_error()

logger = logging.getLogger()

In [3]:
BERT_MODEL_NAME = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=BERT_MODEL_NAME)

train_data_files = DataFiles(
    ["/content/comp-550-project/data/articles/augmentation/train.json"]
)

train_dataset = Dataset(train_data_files, data_format="json")
train_dataloader = DataLoader(train_dataset, batch_size=4)

valid_data_files = DataFiles(
    ["/content/comp-550-project/data/articles/augmentation/validation.json"]
)
valid_dataset = Dataset(valid_data_files, data_format="json")
valid_dataloader = DataLoader(valid_dataset, batch_size=4)

Using custom data configuration default-414426bd25cc89f5
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-414426bd25cc89f5/0.0.0/7334ed11270d48b1ef46b6251d4d612598d73f1cd0c2510f656d05a4ff0d849a)
Using custom data configuration default-c4d7b4c9955a7f53
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-c4d7b4c9955a7f53/0.0.0/7334ed11270d48b1ef46b6251d4d612598d73f1cd0c2510f656d05a4ff0d849a)


In [4]:
# data_loader = DataLoader(train_dataset, batch_size=10)

bert = AutoModel.from_pretrained(BERT_MODEL_NAME)

model = HighwayAugmenter(
    tokenizer=AutoTokenizer.from_pretrained(BERT_MODEL_NAME),
    masking_model=RNN(output_size=2, embeddings_layer=deepcopy(bert.embeddings.word_embeddings)),
    unmasking_model=bert,
    classifier=RNN(output_size=5, embeddings_layer=deepcopy(bert.embeddings.word_embeddings))
)

optimizer = torch.optim.Adam(
    params=[{"params": model.masking_model.parameters()},
            {"params": model.classifier.parameters()}],
    lr=0.001        
)
criterion = WeightedMaskClassificationLoss()

trainer = HighwayAugmenterTrainer(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    train_dataloader=train_dataloader,
    val_dataloader=valid_dataloader,
    logger=logger,
    num_epochs=2,
    log_interval=1
)

trainer.train()

In [6]:
trainer.best_f1_score

0.24324324324324326

In [ ]:
import os
from src.utils.json_utils import write_to_json

PATH = "/home/pavlo/comp-550/comp-550-project/data/articles/augmentation"

with open(os.path.join(PATH, "training_text.txt")) as f:
    training_text = f.readlines()
    training_text = [i.strip() for i in training_text]

with open(os.path.join(PATH, "training_labels.txt")) as f:
    training_labels = f.readlines()
    training_labels = [int(i) for i in training_labels]

# write_to_json(
#     X=training_text,
#     y=training_labels,
#     output_folder_path=PATH,
#     json_name="train.json",

# )
max(training_labels)


4